########################################################
# EXPERIMENT CODE
########################################################

In [6]:
# Initialization Cell for Researcher Metadata

import os
import getpass
import platform
import sys
import uuid
from datetime import datetime

def prompt_if_none(env_key, prompt_text, default_value="unknown"):
    val = os.getenv(env_key)
    if not val:
        try:
            val = input(f"{prompt_text} (default: {default_value}): ").strip() or default_value
        except Exception:
            val = default_value
    return val

def collect_session_metadata(
    prompt_fields=True,
    fixed_role=None,
    fixed_project_id=None,
    fixed_exercise_id=None
):
    metadata = {
        "SESSION_session_id": str(uuid.uuid4()),
        "SESSION_username": os.getenv("JUPYTERHUB_USER", getpass.getuser()),
        "SESSION_timestamp_utc": datetime.utcnow().isoformat(),
        "SESSION_hostname": platform.node(),
        "SESSION_platform": platform.system(),
        "SESSION_os_version": platform.version(),
        "SESSION_python_version": sys.version.split()[0],
    }

    if prompt_fields:
        metadata["SESSION_role"] = fixed_role or prompt_if_none("RESEARCHER_ROLE", "Enter your role", "collaborator")
        metadata["SESSION_project_id"] = fixed_project_id or prompt_if_none("PROJECT_ID", "Enter project ID", "default_project")
    else:
        metadata["SESSION_role"] = fixed_role or os.getenv("RESEARCHER_ROLE", "researcher")
        metadata["SESSION_project_id"] = fixed_project_id or os.getenv("PROJECT_ID", "default_project")

    print("\n📌 Session Metadata:")
    for k, v in metadata.items():
        print(f"  {k}: {v}")

    return metadata


In [7]:

# ============================
# ⚙️ Install Dependencies (if needed )
# ============================
# !pip install mlflow scikit-learn pandas numpy matplotlib seaborn shap requests GitPython
# !pip install --upgrade threadpoolctl
# !pip install setuptools
# !pip install ace_tools 
# !pip install rdflib
# !pip install streamlit-option-menu
# !pip install streamlit-agraph


LIBRARY IMPORTS:

In [43]:
# ============================
# 📦 Standard Library Imports
# ============================
import os
import glob
import io
import json
import time
import ast
import pickle
import platform
import subprocess
from datetime import datetime, timezone
from pprint import pprint
from typing import List, Dict, Any
import xml.etree.ElementTree as ET
import urllib.parse

# ============================
# 📊 Data and Visualization
# ============================
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# ============================
# 🤖 Machine Learning
# ============================
import sklearn
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, label_binarize
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    RocCurveDisplay,
    PrecisionRecallDisplay
)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
# ============================
# 🔬 Experiment Tracking
# ============================
import mlflow
import mlflow.sklearn
from mlflow import MlflowClient

# ============================
# 🌐 Web / API / Networking
# ============================
import requests
from dotenv import load_dotenv

# ============================
# 🧪 Git & Version Control
# ============================
import git
from git import Repo, GitCommandError
import hashlib


# ============================
# 🧠 SHAP for Explainability
# ============================
import shap

# ============================
# 🧬 RDF & Provenance (rdflib)
# ============================
from rdflib import Graph, URIRef, Literal
from rdflib.namespace import PROV, XSD

# ============================
# ⚙️ System Monitoring
# ============================
import psutil


#Dataset metadata!!!

In [26]:
# Define the categorized fields in JSON structure
categorized_fields = {
    "FAIR": [
        "dataset_identifier",
        "dataset_title",
        "dataset_description",
        "dataset_creator",
        "dataset_publisher",
        "dataset_publication_date",
        "dataset_version",
        "dataset_license",
        "dataset_keywords",
        "dataset_access_url",
        "dataset_documentation",
        "metadata_standard",
        "related_resources"
    ],
    "PROV-O": {
        "prov:Entity": "",                       # The dataset being described
        "prov:Activity": "",                     # High-level process involving multiple agents
        "prov:Agent": {
            "dataset_creator": "",
            "database_creator": ""
        },
        "prov:wasGeneratedBy": "",               # Who made the database available
        "prov:used": "",                         # Access URL used as data source
        "prov:wasDerivedFrom": "",               # Original dataset ID
        "prov:wasAttributedTo": "",              # Attribution to dataset author
        "prov:wasAssociatedWith": "",            # Organization releasing the DB
        "prov:startedAtTime": "",                # When dataset was published
        "prov:endedAtTime": "",                  # When DB version was published
        "prov:location": "",                     # Final hosting location
        "prov:role": {
            "dataset_creator_role": "",
            "database_creator_role": ""
        }
    },
    "FAIR4ML": [
        "dataset_dataset_type",
        "columns",
        "target_variable",
        "ml_task",
        "num_samples",
        "data_distribution",
        "known_issues",
        "trainedOn",
        "testedOn",
        "validatedOn",
        "modelRisks",
        "usageInstructions",
        "ethicalLegalSocial"
    ],
    "Internal/DBRepo": [
        "database_identifier",
        "database_title",
        "database_description",
        "database_creator",
        "database_publisher",
        "database_publication_date",
        "database_version",
        "database_schema_public",
        "database_access_url",
        "dataset_version",
        "dataset_id",
        "dataset_name"
        
    ]
}

#Metadata from ZONEDO

In [10]:
import requests

def update_categorized_fields_fair(doi: str, categorized_fields: dict):
    base_url = f"https://api.datacite.org/dois/{doi.lower()}"
    r = requests.get(base_url)
    r.raise_for_status()
    meta = r.json().get("data", {}).get("attributes", {})

    # Extract values
    title = meta.get("titles", [{}])[0].get("title", "info not available")
    creators = [c.get("name", "") for c in meta.get("creators", [])]
    publisher = meta.get("publisher", "info not available")
    pub_year = meta.get("publicationYear", "info not available")
    url = meta.get("url", f"https://doi.org/{doi}")
    
    categorized_fields["FAIR"] = {
        "dataset_identifier": doi,
        "dataset_title": title,
        "dataset_description": meta.get("descriptions", "info not available"),
        "dataset_creator": ", ".join(creators) if creators else "info not available",
        "dataset_publisher": publisher,
        "dataset_publication_date": pub_year,
        "dataset_version": meta.get("version", "info not available"),
        "dataset_license": meta.get("rightsList", "info not available"),
        "dataset_keywords": "info not available",  # DataCite schema doesn't expose keywords directly
        "dataset_access_url": url,
        "dataset_documentation": url,
        "metadata_standard": meta.get("types", {}).get("resourceTypeGeneral", "info not available"),
        "related_resources": url
    }

    categorized_fields["PROV-O"] = {
        "prov:Entity": title,
        "prov:Activity": "Ingestion and Publication",
        "prov:Agent": {
            "dataset_creator": ", ".join(creators) if creators else "info not available",
        },
        "prov:used": url,
        "prov:wasDerivedFrom": doi,
        "prov:wasAttributedTo": ", ".join(creators) if creators else "info not available",
        "prov:startedAtTime": pub_year,
        "prov:role": {
            "dataset_creator_role": "Original Data Author",
            "database_creator_role": "Database Ingestor and Maintainer"
        }
    }


{'FAIR': {'dataset_identifier': '10.24432/C56C76',
  'dataset_title': 'Iris',
  'dataset_description': [],
  'dataset_creator': 'R. A. Fisher',
  'dataset_publisher': 'UCI Machine Learning Repository',
  'dataset_publication_date': 1936,
  'dataset_version': None,
  'dataset_license': [],
  'dataset_keywords': 'info not available',
  'dataset_access_url': 'https://archive.ics.uci.edu/dataset/53',
  'dataset_documentation': 'https://archive.ics.uci.edu/dataset/53',
  'metadata_standard': 'Dataset',
  'related_resources': 'https://archive.ics.uci.edu/dataset/53'},
 'PROV-O': {'prov:Entity': 'Iris',
  'prov:Activity': 'Ingestion and Publication',
  'prov:Agent': {'dataset_creator': 'R. A. Fisher'},
  'prov:used': 'https://archive.ics.uci.edu/dataset/53',
  'prov:wasDerivedFrom': '10.24432/C56C76',
  'prov:wasAttributedTo': 'R. A. Fisher',
  'prov:startedAtTime': 1936,
  'prov:role': {'dataset_creator_role': 'Original Data Author',
   'database_creator_role': 'Database Ingestor and Maintai

In [12]:
#fetch DBREPO metadata

In [13]:

import requests

DB_API = "http://localhost/api/database/{db_id}"
HISTORY_API = "http://localhost/api/database/{db_id}/table/{table_id}/history"

def fetch_db_metadata(db_id: str, table_id: str,selected_version: str) -> dict:
    try:
        # Fetch main DB metadata
        db_url = DB_API.format(db_id=db_id)
        db_response = requests.get(db_url)
        db_response.raise_for_status()
        db_data = db_response.json()
        print(db_data)

        # Fetch table history metadata
        history_url = HISTORY_API.format(db_id=db_id, table_id=table_id)
        history_response = requests.get(history_url)
        if history_response.status_code == 200:
            history_data = history_response.json()
            print(history_data)

            timestamp = history_data[0].get("timestamp") if history_data and isinstance(history_data[0], dict) else "info not available"
        else:
            print(f"Error: Received status code {history_response.status_code}")
            print("Response content:", history_response.text)
            timestamp = "info not available"

        ml_task = "classification"
        categorized_fields[ "PROV-O"] = {
                "prov:Entity": db_data.get("name", "info not available"),
                "prov:Activity": "Ingestion and Publication",
                "prov:Agent": {
                    "dataset_creator": "info not available",  # Not present in DB data
                    "database_creator": db_data.get('owner', {}).get('name', 'info not available')
                },
                "prov:wasGeneratedBy": db_data.get('owner', {}).get('name', 'info not available'),
                "prov:used": db_url,
                "prov:wasDerivedFrom": "info not available",
                "prov:wasAttributedTo": "info not available",
                "prov:wasAssociatedWith": db_data.get('owner', {}).get('name', 'info not available'),
                "prov:startedAtTime": "info not available",
                "prov:endedAtTime": timestamp,
                "prov:location": db_url,
                "prov:role": {
                    "dataset_creator_role": "",
                    "database_creator_role": "Database Ingestor and Maintainer"
                }
            }
        categorized_fields[ "FAIR4ML"] = {
                "dataset_dataset_type": "tabular",
                "columns": db_data.get("columns", "info not available"),
                "target_variable": "",  # TODO
                "ml_task": ml_task,
                "num_samples": "",  # TODO
                "data_distribution": "info not available",  # TODO
                "known_issues": "info not available",  # TODO
                "trainedOn": "info not available",  # TODO
                "testedOn": "info not available",  # TODO
                "validatedOn": "info not available",  # TODO
                "modelRisks": "info not available",  # TODO
                "usageInstructions": "info not available",  # TODO
                "ethicalLegalSocial": "info not available",  # TODO
            }
        categorized_fields[ "Internal/DBRepo"] =  {
                "database_identifier": db_data.get("id", "info not available"),
                "database_title": db_data.get("name", "info not available"),
                "database_description": db_data.get("description", "info not available"),
                "database_creator": db_data.get("owner", {}).get("name", "info not available"),
                "database_publisher": db_data.get("owner", {}).get("name", "info not available"),
                "database_publication_date": timestamp,
                "database_version": db_data.get("is_versioned", "info not available"),
                "database_schema_public": db_data.get("is_schema_public", "info not available"),
                "database_access_url": db_url,
                "dataset_version":selected_version,
                "dataset_id":table_id,
                "dataset_name":next( (t.get("name") for t in db_data.get("tables", []) if t.get("id") == table_id),
                                    "table name not available")

            }
        

        return categorized_fields

    except requests.exceptions.RequestException as e:
        print(f"[⚠️ Error] Failed to fetch DB metadata for {db_id}: {e}")
        return {}


Fetch info needed to fetch metadata:

In [27]:
 #Datasbase info logging
# Step 1: Version → Table ID Mapping
version_to_table_id = {
    "v0": "5315e7da-64fb-4fdb-b493-95b4138c765f",  # Original
    "v1": "926dba08-5078-4f81-a9d1-610b3e122665",                      # Replace with actual UUIDs
    "v2": "743fe377-a4f9-4b1a-9213-af67a6a60078",
    "v3": "6c953963-afc1-4c64-a666-2b8b10e17b95",
    "v4": "92af8331-e56c-466b-898e-b11da87fc129"
}

# Step 2: Prompt User to Choose Version
selected_version = input("Select dataset version (v0(Original), v1(Duplicated), v2(First 100), v3(Shuffled), v4(Normalized)): ").strip().lower()

# Step 3: Validate and Fetch Table ID
if selected_version not in version_to_table_id:
    raise ValueError(f"Invalid version selected: {selected_version}")

selected_table_id = version_to_table_id[selected_version]

# Step 4: Fetch Metadata

db_id = "c3a42d17-42b7-43c9-a504-2363fb4c9c8d"


Select dataset version (v0(Original), v1(Duplicated), v2(First 100), v3(Shuffled), v4(Normalized)):  v3


In [28]:
def log_categorized_fields_to_mlflow(categorized_fields: dict):
    def safe_tag(key, value):
        # MLflow-safe keys (no colons or invalid characters)
        key_clean = key.replace(":", "_").replace("/", "_").replace(" ", "_")
        try:
            mlflow.set_tag(key_clean, json.dumps(value) if isinstance(value, (dict, list)) else str(value))
        except Exception as e:
            print(f"[⚠️ Tag error] {key_clean}: {e}")

    for section, fields in categorized_fields.items():
        if isinstance(fields, dict):
            for key, value in fields.items():
                if isinstance(value, dict):  # e.g. nested like prov:Agent
                    for subkey, subval in value.items():
                        full_key = f"{section}_{key}_{subkey}"
                        safe_tag(full_key, subval)
                else:
                    full_key = f"{section}_{key}"
                    safe_tag(full_key, value)
        else:
            print(f"[⚠️ Skip] {section} → Not a dict: {fields}")


DBREPO INTEGRETION: API call to fetch the dataset for training

In [16]:
# API endpoint URL
API_URL = f"http://localhost/api/database/{db_id}/table/{selected_table_id}/data?size=100000&page=0"

# Define the headers
headers = {
    "Accept": "application/json"  # Specify the expected response format
}

try:
    # Send a GET request to the API with the Accept header
    response = requests.get(API_URL, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        dataset = response.json()
        
        
        print( dataset)
    else:
        print(f"Error: Received status code {response.status_code}")
        print("Response content:", response.text)
       

except requests.exceptions.RequestException as e:
    print(f"Request failed: {e}")


[{'id': '1', 'sepallengthcm': '5.100000000000000000', 'sepalwidthcm': '3.500000000000000000', 'petallengthcm': '1.400000000000000000', 'petalwidthcm': '0.200000000000000000', 'species': 'Iris-setosa'}, {'id': '2', 'sepallengthcm': '4.900000000000000000', 'sepalwidthcm': '3.000000000000000000', 'petallengthcm': '1.400000000000000000', 'petalwidthcm': '0.200000000000000000', 'species': 'Iris-setosa'}, {'id': '3', 'sepallengthcm': '4.700000000000000000', 'sepalwidthcm': '3.200000000000000000', 'petallengthcm': '1.300000000000000000', 'petalwidthcm': '0.200000000000000000', 'species': 'Iris-setosa'}, {'id': '4', 'sepallengthcm': '4.600000000000000000', 'sepalwidthcm': '3.100000000000000000', 'petallengthcm': '1.500000000000000000', 'petalwidthcm': '0.200000000000000000', 'species': 'Iris-setosa'}, {'id': '5', 'sepallengthcm': '5.000000000000000000', 'sepalwidthcm': '3.600000000000000000', 'petallengthcm': '1.400000000000000000', 'petalwidthcm': '0.200000000000000000', 'species': 'Iris-seto

replacing dynamic fetching of data When and if DBREPO isnt running (BACKUP)

In [17]:
# 1. Read the JSON file id the API isnt available this data is saved locally but the data is from the API endpoint
with open("iris_data.json", "r") as f:
    dataset = json.load(f)


Metadata fetching from db repo API CALLS

METADATA ON DATABASE LEVEL

MATADATA FROM: <ns0:OAI-PMH xmlns:ns0="http://www.openarchives.org/OAI/2.0/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.openarchives.org/OAI/2.0/ http://www.openarchives.org/OAI/2.0/OAI-PMH.xsd">

##################################################################
# DATA PREPROCESSING STEPS
###################################################################

STEP 1: LOAD DATASET

STEP2: seperate Dependent and Independent variables and drop unnecessary columns like ID

STEP3: Label Encoding as the target values are class names

# ============================
# 📂 Setup MLflow
# ============================

In [18]:

project_dir = os.getcwd()
mlflow.set_tracking_uri("mlrunlogs/mlflow.db")
experiment_name = input("Enter experiment name for MLflow: ").strip()
mlflow.set_experiment(experiment_name)
# mlflow.sklearn.autolog()

Enter experiment name for MLflow:  test1


<Experiment: artifact_location='file:///C:/Users/reema/REPO/notebooks/RQ_notebooks/mlrunlogs/mlflow.db/730849843658119617', creation_time=1746644031263, experiment_id='730849843658119617', last_update_time=1746644031263, lifecycle_stage='active', name='test1', tags={}>

# ============================
# 🔄 Git Commit Hash for previous commit for metadata
# ============================

In [29]:

repo_dir = "C:/Users/reema/REPO"
previous_commit_repo = git.Repo(repo_dir)
previous_commit_hash = previous_commit_repo.head.object.hexsha

# ============================
# Make threadpoolctl safe so MLflow’s autologger won’t crash ───
# ============================

In [30]:
try:
    import threadpoolctl
    _orig = threadpoolctl.threadpool_info
    def _safe_threadpool_info(*args, **kwargs):
        try:
            return _orig(*args, **kwargs)
        except Exception:
            return []
    threadpoolctl.threadpool_info = _safe_threadpool_info
except ImportError:
    pass  # if threadpoolctl isn’t installed, autolog will skip unsupported versions

# ─── 1) Enable generic autolog (will auto-patch sklearn under the hood) ───
import mlflow
mlflow.autolog(
    log_input_examples=True,
    log_model_signatures=True
)

2025/05/07 21:13:36 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/07 21:13:36 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.


In [31]:
import psutil #TODO, make it as dynamic as possible
import platform
import json
import os

def log_standard_metadata(
    model_name: str,
    model,
    hyperparams: dict,
    acc: float,
    prec: float,
    rec: float,
    f1: float,
    auc: float,
    label_map: dict,
    run_id: str,
    test_size: float,
    random_state: int,
    run_data=None
):
    # ========== FAIR4ML (model-related only) ==========
    mlflow.set_tag("FAIR4ML_intendedUse", "Multiclass classification using RandomForest")
    mlflow.set_tag("FAIR4ML_fineTunedFrom", model_name if "fine-tuned" in model_name.lower() else "None")
    mlflow.set_tag("FAIR4ML_usageInstructions", "Call `.predict()` on a feature-aligned DataFrame")
    mlflow.set_tag("FAIR4ML_ethicalLegalSocial", "No ethical risks; trained on open dataset")
    mlflow.set_tag("FAIR4ML_modelRisks", "May misclassify overlapping species")
    mlflow.set_tag("FAIR4ML_hasCO2eEmissions", "Low; trained locally on a small dataset")
    mlflow.set_tag("FAIR4ML_sharedBy", "Your Name or Organization")
    mlflow.set_tag("FAIR4ML_serializationFormat", "pickle")

    # ========== Croissant structure (minimal) ==========
    mlflow.set_tag("CR_RecordSet", "Feature vectors for model training")
    mlflow.set_tag("CR_Field", "Input features used for training")
    mlflow.set_tag("CR_encodingFormat", "text/csv")

    # ========== MLSEA ==========
    mlflow.set_tag("MLSEA_experimentId", run_id)
    mlflow.set_tag("MLSEA_modelArchitecture", model.__class__.__name__)
    mlflow.set_tag("MLSEA_hyperparameters", json.dumps(ML_EXP_hyperparams))
    mlflow.set_tag("MLSEA_trainingProcedure", f"train_test_split with test_size={test_size}, random_state={random_state}")
    mlflow.set_tag("MLSEA_evaluationMetrics", json.dumps({
        "accuracy": acc,
        "precision_macro": prec,
        "recall_macro": rec,
        "f1_macro": f1,
        "roc_auc": auc
    }))
    preprocessing_info = {
    "label_encoding": label_map,
    "dropped_columns": id_cols,
    "numeric_coercion_attempted": list(X.columns),
    "final_feature_columns": list(X.columns),
    "target_column": target_col,
    "coercion_strategy": "Tried to cast all features to numeric",
    "missing_value_strategy": "None detected in dataset",
    "outlier_detection": "None applied",
    "categorical_encoding": "LabelEncoder for target, none for features",
    "feature_selection": {
        "method": "None",
        "selected_features": list(X.columns)
    },
    "train_test_split": {
        "test_size": test_size,
        "random_state": random_state,
        "stratified": False
    },
    "pipeline_used": False,
    "feature_engineering": "None",
    "preprocessing_version": "v1.0",
    "preprocessing_timestamp": datetime.now().isoformat(),
    "scaling_applied": "None",
    "feature_normalization": False,
    "duplicate_rows_removed": False,
    "sampling_strategy": "None",
    "imbalance_ratio": str(dict(zip(*np.unique(y, return_counts=True))))
    }
     # Hash for reproducibility
    preprocessing_hash = hashlib.sha256(json.dumps(preprocessing_info).encode()).hexdigest()
    mlflow.set_tag("MLSEA_preprocessing_hash", preprocessing_hash)

    mlflow.set_tag("MLSEA_dataPreprocessing", json.dumps(preprocessing_info))
    mlflow.set_tag("MLSEA_modelPath", f"{model_name}.pkl")

    mlflow.set_tag("MLSEA_performanceInterpretation", (
        "Model performs well with F1 macro > 0.90. "
        "ROC AUC confirms balanced performance across all classes."
    ))

    # Git commit hash (optional, safe fallback)
    try:
        from subprocess import check_output
        sha = check_output(["git", "rev-parse", "HEAD"], text=True).strip()
    except Exception:
        sha = "unknown"
    mlflow.set_tag("MLSEA_trainingCodeSnapshot", sha)

    mlflow.set_tag("MLSEA_previousModelRunId", "None")
    mlflow.set_tag("MLSEA_improvedFrom", "baseline_randomforest_2024")

    # Compute environment
    compute_env = {
        "os": f"{platform.system()} {platform.release()}",
        "cpu": platform.processor(),
        "ram_gb": round(psutil.virtual_memory().total / (1024 ** 3), 2),
        "python_version": platform.python_version(),
        "sklearn_version": sklearn.__version__,
        "pandas_version": pd.__version__,
        "numpy_version": np.__version__,
    }
    mlflow.set_tag("MLSEA_computeEnvironment", json.dumps(compute_env))

    mlflow.set_tag("MLSEA_performanceInterpretation", (
        "Model performs well with F1 macro > 0.90. "
        "ROC AUC confirms balanced performance across all classes."
    ))

    # Justifications (already captured as justification_X → re-tag under mlsea)
    if run_data is not None:
        for key, val in run_data.tags.items():
            if key.startswith("justification_"):
                new_key = "MLSEA_justification_" + key.replace("justification_", "")
                mlflow.set_tag(new_key, val)



In [41]:

def generate_reproducibility_txt_log(
    model_name: str,
    dataset_name: str,
    dataset_version: str,
    hyperparams: dict,
    metrics: dict,
    git_commit: str,
    run_id: str
) -> str:
    """Generate a nicely structured reproducibility log and upload to MLflow."""

    def clean_values(d):
        """Convert np.float types to native floats for clean YAML."""
        return {k: float(v) if isinstance(v, (np.float32, np.float64)) else v for k, v in d.items()}

    # ⏱️ Timestamp for tracking
    timestamp = datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC")

    # 🎯 Build log structure
    repro_data = {
        "📌 Model Details": {
            "Model Name": model_name,
            "Dataset Name": dataset_name,
            "Dataset Version": dataset_version,
            "Run ID": run_id,
            "Timestamp": timestamp,  


            
        },
        "🛠️ Hyperparameters": clean_values(hyperparams),

        
        "📈 Metrics": clean_values(metrics),

        
        "🔗 Git Info": {
            "Commit Hash": git_commit,
            "Reproduce With": f"git checkout {git_commit}"
        },

        
        "🚀 Reproduction Guide": [
            
            "1. Clone the repo and checkout the commit:",
            
            f"   git checkout {git_commit}",
            
            "2. Load and preprocess the dataset exactly as during training.",
            
            "3. Load the model using MLflow:",

            f"   mlflow.sklearn.load_model('runs:/{run_id}/model')",
            
            "4. Run inference or evaluation using the same pipeline/script."
        ]
    }

    # 🗂️ Save directory
    save_dir = os.path.join("MODEL_PROVENANCE", model_name)
    os.makedirs(save_dir, exist_ok=True)
    txt_path = os.path.join(save_dir, f"{model_name}_reproducibility.txt")

    # 📝 Dump as YAML-formatted text
    with open(txt_path, "w", encoding="utf-8") as f:
        yaml.dump(repro_data, f, allow_unicode=True, sort_keys=False, width=100)


    return txt_path


In [23]:
mlflow.end_run()

# ============================
# 🚀 Start MLflow Run 
# ============================

In [44]:
import hashlib

with mlflow.start_run() as run:
    # Start time
    session_metadata = collect_session_metadata(prompt_fields=True) #Logging session metadta
    mlflow.log_params(session_metadata)
    
    update_categorized_fields_fair("10.24432/C56C76", categorized_fields) #dataset related metadata logging 

    start_time = datetime.now().isoformat()
    mlflow.set_tag("PROV_startedAtTime", start_time) #start time

#################################################
# Justification LOGGER
#################################################

    
    # ── 2) Load into a DataFrame ─────────────────────────────────────────────────
    df = pd.DataFrame(dataset)  # Convert the dataset (likely a list of dicts or 2D structure) into a pandas DataFrame
    count_start = df.shape[0]   # Record the original number of rows
    before_hashes = set(df.astype(str).apply(lambda row: hash(tuple(row)), axis=1))  
    # Create a set of hashes for each row (as string) to detect duplicates or track identity


    
    # ── 3) Extract target variable info ──────────────────────────────────────────
    target_col = df.columns[-1]  # Assume the last column is the target/label (e.g., "species")
    
    categorized_fields["FAIR4ML"] = {
        "target_variable": target_col,   # Store the name of the target variable in metadata
        "num_samples": df.shape[0]       # Store sample count for provenance/tracking
    }


    
    # ── 4) Separate features and labels ─────────────────────────────────────────
    y = df[target_col]  # Extract target labels as a Series (used for model training)
    
    X = df.drop(columns=[target_col])  # Drop the target column to isolate features


    
    # ── 5) Drop ID columns (case-insensitive) ───────────────────────────────────
    id_cols = [c for c in X.columns if c.lower() == "id"]  # Look for columns named "id" (not case-sensitive)
    if id_cols:
        X = X.drop(columns=id_cols)  # Remove ID column(s) if found to avoid data leakage


    
    # ── 6) Convert columns to numeric where possible ────────────────────────────
    for c in X.columns:
        try:
            X[c] = pd.to_numeric(X[c])  # Try converting column to numeric (e.g., float, int)
        except Exception:
            pass  # If conversion fails (e.g., for categorical strings), leave it as-is


    
    # ── 7) Print diagnostic info ────────────────────────────────────────────────
    print("ML_EXP_Shapes:", X.shape, y.shape)  # Output the shape of feature matrix and labels


    
    # ── 8) Encode class labels numerically ──────────────────────────────────────
    le = LabelEncoder()       # Initialize label encoder
    y = le.fit_transform(y)   # Convert string/categorical labels to integers (e.g., ["setosa", "versicolor"] → [0, 1])
    
    print("ML_EXP_Classes:", le.classes_)  # Print original label classes for reference

    # ── 4) Cast feature columns to numeric where possible ─────────────────────────
    for col in X.columns:
        try:
            X[col] = pd.to_numeric(X[col])   # no errors="ignore"
        except ValueError:
            # if it can’t be cast, just leave it as-is
            pass


            
    # ── 5) Drop any “id” column (case-insensitive) ────────────────────────────────
    dropped = [c for c in X.columns if c.lower() == "id"]
    X = X.drop(columns=dropped, errors="ignore")
    # ── Row Count Comparison ──
    count_end = df.shape[0]
    
    after_hashes  = set(df.astype(str).apply(lambda row: hash(tuple(row)), axis=1))
    
    n_insert = len(after_hashes - before_hashes)
    n_delete = len(before_hashes - after_hashes)
    def log_with_justification(log_func, key, value, context=""):
        """
        Log a parameter/metric/tag using `log_func` and ask for justification via console.
        """
        log_func(key, value)
        print(f"\n📝 Justification for `{key}` ({context})")
        user_reason = input("→ Why did you choose this value? ")
        mlflow.set_tag(f"justification_{key}", user_reason or "No justification provided")

    def log_justification(key: str, question: str):
        print(f"\n📝 Justification for `{key}`")
        user_reason = input(f"→ {question} ")
        mlflow.set_tag(f"justification_{key}", user_reason or "No justification provided")



    db_meta = fetch_db_metadata(db_id,selected_table_id,selected_version) #Extended logging
   
    # provenance tags
    mlflow.set_tag("Internal_DBRepo_table_last_modified", categorized_fields["Internal/DBRepo"]["database_publication_date"])

    # row-count metrics
    mlflow.log_metric("Internal_DBRepo_row_count_start", count_start)
    mlflow.log_metric("Internal_DBRepo_row_count_end",   count_end)

    # change-event metrics
    mlflow.log_metric("Internal_DBRepo_num_inserts", n_insert)
    mlflow.log_metric("Internal_DBRepo_num_deletes", n_delete)
    
    # 2) Capture raw metadata
    mlflow.set_tag("Internal_DBRepo_data_source", API_URL)
    mlflow.log_param("Internal_DBRepo_retrieval_time", datetime.utcnow().isoformat())
    mlflow.log_param("Internal_DBRepo_n_records", len(df))
    mlflow.log_param("Internal_DBRepo_columns_raw", df.columns.tolist())
    mlflow.log_param("Internal_DBRepo_dropped_columns", id_cols)

    # 4) Post‐processing metadata
    mlflow.log_param("Internal_DBRepo_n_features_final", X.shape[1])
    mlflow.log_param("Internal_DBRepo_feature_names", X.columns.tolist())
    mlflow.set_tag("Internal_DBRepo_target_name", y)

    # Label encoding
    label_map = {int(idx): cls for idx, cls in enumerate(le.classes_)}
    
    # Save to an in-memory file
    buffer = io.StringIO()
    json.dump(label_map, buffer, indent=2)
    buffer.seek(0)
    
    # Log it to MLflow
    mlflow.log_text(buffer.getvalue(), artifact_file="label_mapping.json")
   
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_name = f"RandomForest_Iris_v{ts}"
    mlflow.set_tag("ML_EXP_model_name",model_name)
    
    train_start_ts = datetime.now().isoformat()
    mlflow.set_tag("ML_EXP_training_start_time", train_start_ts)

    # test_size    = 0.2
    # random_state = 42
    # Prompt user for test size
    try:
        test_size = float(input("Enter test size (e.g., 0.2 for 20% test set): "))
    except ValueError:
        print("Invalid input. Defaulting to 0.2")
        test_size = 0.2
    
    # Prompt user for random seed
    try:
        random_state = int(input("Enter random seed (e.g., 42): "))
    except ValueError:
        print("Invalid input. Defaulting to 42")
        random_state = 42
    
    # 📈 Model Training
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # ── 2) Log dataset split params ──
    mlflow.log_param("ML_EXP_test_size", test_size)
    mlflow.log_param("ML_EXP_random_state", random_state)
    mlflow.log_param("ML_EXP_n_train_samples", X_train.shape[0])
    mlflow.log_param("ML_EXP_n_test_samples",  X_test.shape[0])
    mlflow.log_param("ML_EXP_n_features",      X_train.shape[1])
    mlflow.log_param("ML_EXP_n_test_samples",  X_test.shape[0])
    mlflow.log_param("ML_EXP_n_features",      X_train.shape[1])

    #  # 1) Define a more complex hyperparameter dict
    # ML_EXP_hyperparams = {
    #     "n_estimators":       200,
    #     "criterion":          "entropy",
    #     "max_depth":          12,
    #     "min_samples_split":  5,
    #     "min_samples_leaf":   2,
    #     "max_features":       "sqrt",
    #     "bootstrap":          True,
    #     "oob_score":          False,
    #     "class_weight":       None,
    #     "random_state":       42,
    #     "verbose":            1,
    #     "n_jobs":             -1
    # }
      # 1) Define a more complex hyperparameter dict
    ML_EXP_hyperparams = {
        "n_estimators":       100,
        "criterion":          "entropy",
        "max_depth":          10,
        "min_samples_split":  3,
        "min_samples_leaf":   1,
        "max_features":       "sqrt",
        "bootstrap":          True,
        "oob_score":          True,
        "class_weight":       None,
        "random_state":       26,
        "verbose":            1,
        "n_jobs":             -1
    }
    
    # # 2) Log them ALL at once
    # model = RandomForestClassifier(**ML_EXP_hyperparams)
 
    
    # Available models
    available_models = {
        "random_forest": RandomForestClassifier,
        "decision_tree": DecisionTreeClassifier,
        "logistic_regression": LogisticRegression,
        "knn": KNeighborsClassifier,
        "svm": SVC,
        "gradient_boosting": GradientBoostingClassifier
    }
    
    # Prompt user to choose
    print("Choose a model to train:")
    for i, name in enumerate(available_models.keys()):
        print(f"{i + 1}. {name}")
    
    choice = int(input("Enter model number (e.g., 1 for random_forest): "))
    selected_key = list(available_models.keys())[choice - 1]
    selected_model_class = available_models[selected_key]
    
    # Initialize with your hyperparams (define this dict earlier)
    model = selected_model_class(**ML_EXP_hyperparams)

    for key, val in ML_EXP_hyperparams.items():
        log_with_justification(mlflow.log_param, key, val, context="Hyperparameter configuration")

    # Prompt for and log justifications for high-level modeling decisions
    log_justification("ML_EXP_model_choice", "Why did you choose RandomForestClassifier for this task?")
    log_justification("ML_EXP_target_variable", "Why did you choose this column as the prediction target?")
    log_justification("ML_EXP_test_split", "Why this train/test ratio (e.g., 80/20)?")
    log_justification("ML_EXP_metric_choice", "Why accuracy/f1/ROC-AUC as your evaluation metric?")
    log_justification("ML_EXP_threshold_accuracy", "Was there a threshold for accurary, why?")
    log_justification("ML_EXP_dataset_version", "Why use this specific dataset version?")
    log_justification("ML_EXP_drop_column_X", "Why drop any specific columns from the dataset?")
    log_justification("ML_EXP_experiment_name", "Any context behind this experiment name or setup?")



    
    model.fit(X_train, y_train)

    train_end_ts = datetime.now().isoformat()
    mlflow.set_tag("ML_EXP_training_end_time", train_end_ts)

     # ── 6) Predict & log metrics ──
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba, multi_class="ovr")
    prec = precision_score(y_test, y_pred, average="macro")
    rec  = recall_score(y_test,    y_pred, average="macro")
    f1   = f1_score(y_test,      y_pred, average="macro")
    
    mlflow.log_metric("ML_EXP_precision_macro", prec)
    mlflow.log_metric("ML_EXP_recall_macro",    rec)
    mlflow.log_metric("ML_EXP_f1_macro",        f1)
    mlflow.log_metric("ML_EXP_accuracy", acc)
    mlflow.log_metric("ML_EXP_roc_auc",   auc)

    # ✅ Log Environment Automatically
    mlflow.log_params({
        "python_version": platform.python_version(),
        "os_platform": f"{platform.system()} {platform.release()}",
        "sklearn_version": sklearn.__version__,
        "pandas_version": pd.__version__,
        "numpy_version": np.__version__,
        "matplotlib_version": matplotlib.__version__,
        "seaborn_version": sns.__version__,
        "shap_version": shap.__version__,
    })

 
    mlflow.set_tag("ML_EXP_notebook_name", "RQ1_2.ipynb")

    # ✅ Dataset Metadata Tags
    mlflow.set_tag("ML_EXP_dataset_name", categorized_fields[ "Internal/DBRepo"]["dataset_name"]) 
    mlflow.set_tag("ML_EXP_dataset_version", selected_version) 
    mlflow.set_tag("ML_EXP_dataset_id", selected_table_id) 


    # ─── 2) Create a folder for this run’s plots ───
    plot_dir = os.path.join("ML_EXP_plots", model_name)
    os.makedirs(plot_dir, exist_ok=True)

    # 1) Feature Importance Bar Chart
    importances = model.feature_importances_
    try:
        feature_names = X_train.columns
    except AttributeError:
        feature_names = [f"f{i}" for i in range(X_train.shape[1])]
    fi_path = os.path.join(plot_dir, "feature_importances.png")

    plt.figure(figsize=(8, 6))
    sns.barplot(x=importances, y=feature_names)
    plt.title("Feature Importances")
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.tight_layout()
    plt.savefig(fi_path)
    mlflow.log_artifact(fi_path)
    plt.close()


# 2) Multi-class ROC Curves
# Binarize labels for one-vs-rest
    classes = np.unique(y_test)
    y_test_bin = label_binarize(y_test, classes=classes)
    
    for idx, cls in enumerate(classes):
        disp = RocCurveDisplay.from_predictions(
            y_test_bin[:, idx], 
            y_proba[:, idx],
            name=f"ROC for class {cls}"
        )
        roc_path = os.path.join(plot_dir, f"roc_curve_cls_{cls}.png")
        disp.figure_.savefig(roc_path)
        mlflow.log_artifact(roc_path)
        plt.close(disp.figure_)


    # 3) Multi-class Precision-Recall Curves
    for idx, cls in enumerate(classes):
        disp = PrecisionRecallDisplay.from_predictions(
            y_test_bin[:, idx], 
            y_proba[:, idx],
            name=f"PR curve for class {cls}"
        )
        pr_path = os.path.join(plot_dir, f"pr_curve_cls_{cls}.png")
        disp.figure_.savefig(pr_path)
        mlflow.log_artifact(pr_path)
        plt.close(disp.figure_)
        
    # ✅ Confusion Matrix Plot
    cm_path = os.path.join(plot_dir, "confusion_matrix.png")

    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.savefig(cm_path)
    mlflow.log_artifact(cm_path)

    # ✅ SHAP Summary
    shap_path = os.path.join(plot_dir, "shap_summary.png")
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test)
    shap.summary_plot(shap_values, X_test, show=False)
    plt.savefig(shap_path)
    mlflow.log_artifact(shap_path)


    client = MlflowClient()
    run_data = client.get_run(run.info.run_id).data
    
    log_standard_metadata(
    model_name=model_name,
    model=model,
    hyperparams=ML_EXP_hyperparams,
    acc=acc,
    prec=prec,
    rec=rec,
    f1=f1,
    auc=auc,
    label_map=label_map,
    run_id=run.info.run_id,
    test_size=test_size,
    random_state=random_state,
    run_data=run_data # captured via client.get_run(run.info.run_id).data
    )
    # ─── 1) Build a .pkl filename (you can include your model_name for clarity)
    pkl_path = f"Trained_models/{model_name}.pkl"
    
    # ─── 2) Serialize your trained model to disk
    with open(pkl_path, "wb") as f:
        pickle.dump(model, f)
    
    # ─── 3) Log that pickle file as an MLflow artifact
    #     It will appear under Artifacts → models/RandomForest_Iris_vYYYYMMDD_HHMMSS.pkl
    mlflow.log_artifact(pkl_path, artifact_path=model_name)
        
    def get_latest_commit_hash(repo_path="."):
        # returns the full SHA of HEAD
        res = subprocess.run(
            ["git", "-C", repo_path, "rev-parse", "HEAD"],
            capture_output=True, text=True, check=True)
        
        return res.stdout.strip()

    def get_remote_url(repo_path=".", remote="origin"):
        # returns something like git@github.com:user/repo.git or https://...
        res = subprocess.run(
            ["git", "-C", repo_path, "config", "--get", f"remote.{remote}.url"],
            capture_output=True, text=True, check=True
        )
        return res.stdout.strip()
    
    def make_commit_link(remote_url, commit_hash):
        # handle GitHub/GitLab convention; strip “.git” if present
        base = remote_url.rstrip(".git")
        # if SSH form (git@github.com:owner/repo), convert to https
        if base.startswith("git@"):
            base = base.replace(":", "/").replace("git@", "https://")
        return f"{base}/commit/{commit_hash}"

    
    def simple_commit_and_push_and_log(repo_path=".", message="Auto commit", remote="origin", branch="main"):
    # 1) Check for changes
        status = subprocess.run(
            ["git", "-C", repo_path, "status", "--porcelain"],
            capture_output=True, text=True
        )
        if not status.stdout.strip():
            print("🟡 No changes to commit.")
            return None, None
    
        # 2) Stage everything
        add = subprocess.run(
            ["git", "-C", repo_path, "add", "--all"],
            capture_output=True, text=True
        )
        if add.returncode:
            print("❌ git add failed:\n", add.stderr)
            return None, None
    
        # 3) Commit
        commit = subprocess.run(
            ["git", "-C", repo_path, "commit", "-m", message],
            capture_output=True, text=True
        )
        if commit.returncode:
            print("❌ git commit failed:\n", commit.stderr)
            return None, None
        print("✅ Commit successful.")
    
        # 4) Push
        push = subprocess.run(
            ["git", "-C", repo_path, "push", "-u", remote, branch],
            capture_output=True, text=True
        )
        if push.returncode:
            print("❌ git push failed:\n", push.stderr)
        else:
            print("🚀 Push successful.")
    
        # 5) Retrieve hash & remote URL
        sha = get_latest_commit_hash(repo_path)
        url = get_remote_url(repo_path, remote)
        link = make_commit_link(url, sha)
    
        return sha, link
    
      
    sha, link = simple_commit_and_push_and_log(
        repo_path=".",
        message="Auto commit after successful training"
    )
    def get_git_commit_hash():
        """Returns the current Git commit hash."""
        try:
            commit_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD']).decode('utf-8').strip()
        except Exception as e:
            print(f"Error fetching Git commit hash: {e}")
            return None
        return commit_hash
    
    def get_version_tag_for_commit(commit_hash, known_tags=None):
        """
        Ask user to assign a version tag if it's not already known.
    
        Parameters:
        - commit_hash (str): The Git commit hash.
        - known_tags (dict): Optional dict mapping known commit hashes to version tags.
    
        Returns:
        - tuple of (commit_hash, version_tag)
        """
        known_tags = known_tags or {}
    
        version_tag = known_tags.get(commit_hash, "untagged")
    
        if version_tag == "untagged":
            print(f"⚠️ Commit {commit_hash[:8]} is not tagged with a version.")
            user_input = input("🔖 Enter version tag for this commit (or press Enter to skip): ").strip()
            version_tag = user_input if user_input else "untagged"
    
        return commit_hash, version_tag
    # commit = get_git_commit_hash()
    commit, version_tag = get_version_tag_for_commit(sha)

    # Add to JSON or MLflow
    mlflow.set_tag("GIT_code_version", version_tag)
    def get_git_author():
        name = subprocess.check_output(["git", "config", "user.name"]).decode().strip()
        email = subprocess.check_output(["git", "config", "user.email"]).decode().strip()
        return name, email
    name, email=get_git_author()
    mlflow.set_tag("GIT_user", name)
    mlflow.set_tag("GIT_user_email", email)

    if sha and link:
        diff_text = subprocess.check_output(
            ["git", "-C", ".", "diff", previous_commit_hash, sha],
            encoding="utf-8",
            errors="ignore"    # or "replace"
        )
                
        # 1) Get your repo’s remote URL and normalize to HTTPS
        remote_url = subprocess.check_output(
            ["git", "config", "--get", "remote.origin.url"],
            text=True
        ).strip().rstrip(".git")
        if remote_url.startswith("git@"):
            # git@github.com:owner/repo.git → https://github.com/owner/repo
            remote_url = remote_url.replace(":", "/").replace("git@", "https://")
        
        # 2) Build commit URLs
        previous_commit_url  = f"{remote_url}/commit/{previous_commit_hash}"
        current_commit_url = f"{remote_url}/commit/{sha}"
        diff_data = {
            "GIT_previous_commit":  previous_commit_hash,
            "GIT_previous_commit_url":previous_commit_url,
            "GIT_current_commit_url":current_commit_url,
            "GIT_current_commit": sha,
            "GIT_diff": diff_text
        }
        mlflow.log_dict(
            diff_data,
            artifact_file="GIT_commit_diff.json"
        )
        mlflow.set_tag("GIT_previous_commit_hash", previous_commit_hash)
        mlflow.set_tag("GIT_current_commit_hash", sha)
        mlflow.set_tag("GIT_current_commit_url", link) 


    client   = MlflowClient()
    run_id    = run.info.run_id
    run_info  = client.get_run(run_id).info
    run_data  = client.get_run(run_id).data
    
    # 1) params, metrics, tags
    params  = dict(run_data.params)
    metrics = dict(run_data.metrics)
    tags    = dict(run_data.tags)

    # (4) List artifacts under a specific subfolder
    run_meta     = client.get_run(run_id).info
    artifact_uri = run_meta.artifact_uri  # base URI for all artifacts
    
    artifact_meta = []

    def _gather(path=""):
            for af in client.list_artifacts(run_id, path):
                if af.is_dir:
                    _gather(af.path)
                    continue
        
                rel_path = af.path
                lower = rel_path.lower()
        
                if lower.endswith((".json", ".txt", ".patch")):
                    artifact_meta.append({"path": rel_path, "type": "text"})
                elif lower.endswith((".png", ".jpg", ".jpeg", ".svg")):
                    artifact_meta.append({"path": rel_path, "type": "image"})
                else:
                    artifact_meta.append({"path": rel_path, "type": "other"})
    # Run the gather
    _gather()
     
   

    # 1) Determine notebook directory (where your .ipynb lives)
    notebook_dir = os.getcwd()
    
   
   
    
    repro_txt_path = generate_reproducibility_txt_log(
        model_name=model_name,
        dataset_name="Iris",
        dataset_version="1.0.0",
        hyperparams=ML_EXP_hyperparams,
        metrics={
            "accuracy": acc,
            "f1_macro": f1,
            "precision_macro": prec,
            "recall_macro": rec,
            "roc_auc": auc
        },
        git_commit=sha,
        run_id=run.info.run_id
    )
    # mlflow.log_text('Reproducibility TXT logged at', repro_txt_path)
    with open(repro_txt_path, "r", encoding="utf-8") as f:
        content = f.read()
    
        mlflow.log_text(content, artifact_file="run_summaries/repro_log.txt")

    # ✅ Save both YAML and JSON summary to disk
    summary_dir = os.path.join(os.getcwd(), "MODEL_PROVENANCE", model_name)
    os.makedirs(summary_dir, exist_ok=True)
    
    
 

    summary = {
        "ML_EXP_run_id":         run_id,
        "ML_EXP_run_name": run_info.run_name,
        "ML_EXP_experiment_id":  run_info.experiment_id,
        "ML_EXP_start_time":     run_info.start_time,
        "ML_EXP_end_time":       run_info.end_time,
        "ML_EXP_params":         params,
        "ML_EXP_metrics":        metrics,
        "ML_EXP_tags":           tags,
        "ML_EXP_artifacts":      artifact_meta
    }
    
    log_categorized_fields_to_mlflow(categorized_fields)

    # Run summary JSON
    summary_filename = f"{model_name}_run_summary.json"
    summary_local_path = os.path.join(summary_dir, summary_filename)
    with open(summary_local_path, "w", encoding="utf-8") as f:
        json.dump(summary, f, indent=2)
    
    # ✅ Log summary JSON
    mlflow.log_artifact(summary_local_path, artifact_path="run_summaries")
    print("📁 Run summary JSON logged at:", summary_local_path)
    
    # ✅ End MLflow run
    end_time = datetime.now().isoformat()
    mlflow.set_tag("PROV_endedAtTime", end_time)
    mlflow.end_run()

Enter your role (default: collaborator):  
Enter project ID (default: default_project):  



📌 Session Metadata:
  session_id: 5e10e85d-80ba-4963-a179-be75ad1f9a08
  username: reema
  timestamp_utc: 2025-05-07T19:34:17.580518
  hostname: Purplish
  platform: Windows
  os_version: 10.0.26100
  python_version: 3.11.5
  role: collaborator
  project_id: default_project
ML_EXP_Shapes: (150, 4) (150,)
ML_EXP_Classes: ['Iris-setosa' 'Iris-versicolor' 'Iris-virginica']
{'id': 'c3a42d17-42b7-43c9-a504-2363fb4c9c8d', 'name': 'Iris', 'description': None, 'tables': [{'id': '92af8331-e56c-466b-898e-b11da87fc129', 'name': 'Iris_v4', 'alias': None, 'identifiers': [], 'owner': {'id': '82520286-97b2-103f-8282-b1a6f55cd3ef', 'username': 'reema', 'name': 'Reema George Dass', 'orcid': None, 'qualified_name': 'Reema George Dass — @reema', 'given_name': 'Reema George', 'family_name': 'Dass'}, 'description': 'Iris version 4', 'columns': [{'id': '20e5f0c3-2651-4ca6-b7ee-233b2975662c', 'name': 'Id', 'alias': None, 'size': None, 'd': None, 'mean': 76, 'median': 76, 'concept': None, 'unit': None, 'desc

Enter test size (e.g., 0.2 for 20% test set):  0.2
Enter random seed (e.g., 42):  42


Choose a model to train:
1. random_forest
2. decision_tree
3. logistic_regression
4. knn
5. svm
6. gradient_boosting


Enter model number (e.g., 1 for random_forest):  1


...

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
C:\Users\reema\anaconda3\Lib\site-packages\shap\plots\_beeswarm.py:1153: UserWarning: The figure layout has changed to tight
  pl.tight_layout()
C:\Users\reema\anaconda3\Lib\site-packages\shap\plots\_beeswarm.py:761: UserWarning: The figure layout has changed to tight
  pl.tight_layout(pad=0, w_pad=0, h_pad=0.0)


✅ Commit successful.
🚀 Push successful.
⚠️ Commit edb79b9a is not tagged with a version.


🔖 Enter version tag for this commit (or press Enter to skip):  v3


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\reema\\AppData\\Local\\Temp\\tmpkj_iyz2q\\MODEL_PROVENANCE\\RandomForest_Iris_v20250507_213421\\RandomForest_Iris_v20250507_213421_reproducibility.txt'

what does it create: 
lable_mapping in the current dir
provenence file :REPO/notebooks/RQ_notebooks/MODEL_PROVENANCE/RandomForest_Iris_v20250425_120045_run_summary.json
plots based on run:REPO/notebooks/RQ_notebooks/plots/RandomForest_Iris_v20250425_120045/shap_summary.png
mlrun:REPO/notebooks/RQ_notebooks/mlrunlogs/mlflow.db/615223710259862608/5d1fa0fc65af47128f3200628b1afaea
trained model:REPO/notebooks/RQ_notebooks/Trained_models/RandomForest_Iris_v20250425_120852.pkl

1. Standards-compliant export (JSON-LD + Turtle)
I already have your plain run_summary.json , wrap it in a JSON-LD context that maps your fields into PROV-O terms, then use rdflib to emit Turtle:

In [ ]:
import json
import pandas as pd

# Load the JSON file
json_path = "/mnt/data/REPO/notebooks/RQ_notebooks/MODEL_PROVENANCE/RandomForest_Iris_v20250425_125653/RandomForest_Iris_v20250425_125653_run_summary.json"
with open(json_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Extract justification tags
justifications = {
    k: v for k, v in data.get("tags", {}).items()
    if k.startswith("justification_")
}

# Create a DataFrame
justification_df = pd.DataFrame([
    {"Decision": k.replace("justification_", ""), "Justification": v}
    for k, v in justifications.items()
])

import ace_tools as tools; tools.display_dataframe_to_user(name="Researcher Justifications", dataframe=justification_df)


In [66]:

def iso8601(ms):
    """Convert milliseconds since epoch to ISO8601 UTC."""
    return datetime.fromtimestamp(ms / 1000, tz=timezone.utc).isoformat()

for json_path in glob.glob("MODEL_PROVENANCE/*/*_run_summary.json"):
    basename   = os.path.basename(json_path)
    model_name = basename.rsplit("_run_summary.json", 1)[0]

    with open(json_path, "r", encoding="utf-8") as f:
        summary = json.load(f)

    #–– Minimal override context: keep all your flat fields as-is,
    #–– and only map the actual PROV terms to their IRIs.
    ctx = {
        # keep these flat
        "run_id":       { "@id": "run_id" },
        "run_name":     { "@id": "run_name" },
        "experiment_id":{ "@id": "experiment_id" },
        "params":       { "@id": "params" },
        "metrics":      { "@id": "metrics" },
        "artifacts":    { "@id": "artifacts" },
        "tags":         { "@id": "tags" },

        # provenance namespace
        "prov": "http://www.w3.org/ns/prov#",
        "xsd":  "http://www.w3.org/2001/XMLSchema#",

        # map your timestamp fields into PROV
        "start_time": { "@id": "prov:startedAtTime", "@type": "xsd:dateTime" },
        "end_time":   { "@id": "prov:endedAtTime",   "@type": "xsd:dateTime" },

        # PROV-used/generated
        "used":      { "@id": "prov:used",      "@type": "@id" },
        "generated": { "@id": "prov:generated", "@type": "@id" },

        # JSON-LD boilerplate
        "@id":   "@id",
        "@type": "@type"
    }

    #–– Build JSON-LD document, re-using your original keys verbatim
    doc = {
        "@context":      ctx,
        "run_id":        summary["run_id"],
        "run_name":      summary.get("run_name"),
        "experiment_id": summary.get("experiment_id"),
        "params":        summary.get("params", {}),
        "metrics":       summary.get("metrics", {}),
        "artifacts":     summary.get("artifacts", []),
        "tags":          summary.get("tags", {}),

        # PROV fields:
        "start_time": iso8601(summary["start_time"])
    }

    if summary.get("end_time") is not None:
        doc["end_time"] = iso8601(summary["end_time"])

    # for used/generated, just point at your dataset/model URIs
    # (or blank-node them if you prefer richer structure)
    doc["used"] = summary.get("tags", {}).get("dataset_uri") or []
    doc["generated"] = [
        art.get("uri") or art.get("path")
        for art in summary.get("artifacts", [])
    ]

    #–– write JSON-LD
    out_jsonld = os.path.join("MODEL_PROVENANCE", model_name, f"{model_name}.jsonld")
    with open(out_jsonld, "w", encoding="utf-8") as f:
        json.dump(doc, f, indent=2)

    #–– parse & serialize to Turtle
    g = Graph().parse(data=json.dumps(doc), format="json-ld")
    out_ttl = os.path.join("MODEL_PROVENANCE", model_name, f"{model_name}.ttl")
    g.serialize(destination=out_ttl, format="turtle")

    print(f"Converted {basename} → {os.path.basename(out_jsonld)}, {os.path.basename(out_ttl)}")



Converted RandomForest_Iris_v20250425_121328_run_summary.json → RandomForest_Iris_v20250425_121328.jsonld, RandomForest_Iris_v20250425_121328.ttl


This code programatically, finds diff between generated Json file and created JsonLD and .TTL file to make it easier to understand if there is any discrepency

In [67]:


def load_as_dict(path):
    if path.endswith((".ttl", ".turtle")):
        g = Graph()
        g.parse(path, format="turtle")
        # normalize to JSON-LD dict
        return json.loads(g.serialize(format="json-ld", indent=2))
    else:
        with open(path, encoding="utf-8") as f:
            return json.load(f)

def compare_json(a, b, path=""):
    diffs = []
    if isinstance(a, dict) and isinstance(b, dict):
        all_keys = set(a) | set(b)
        for k in all_keys:
            new_path = f"{path}/{k}" if path else k
            if k not in a:
                diffs.append({"path": new_path, "type": "added",   "a": None,    "b": b[k]})
            elif k not in b:
                diffs.append({"path": new_path, "type": "removed", "a": a[k],   "b": None})
            else:
                diffs.extend(compare_json(a[k], b[k], new_path))
    elif isinstance(a, list) and isinstance(b, list):
        for i, (ia, ib) in enumerate(zip(a, b)):
            diffs.extend(compare_json(ia, ib, f"{path}[{i}]"))
        # handle length mismatches
        if len(a) < len(b):
            for i in range(len(a), len(b)):
                diffs.append({"path": f"{path}[{i}]", "type": "added",   "a": None,  "b": b[i]})
        elif len(a) > len(b):
            for i in range(len(b), len(a)):
                diffs.append({"path": f"{path}[{i}]", "type": "removed", "a": a[i],  "b": None})
    else:
        if a != b:
            diffs.append({"path": path, "type": "changed", "a": a, "b": b})
    return diffs


# Define base directory
base_dir = os.path.join("MODEL_PROVENANCE", model_name)

# Build full paths for the files to compare
summary_json    = os.path.join(base_dir, f"{model_name}_run_summary.json")
turtle_file     = os.path.join(base_dir, f"{model_name}.ttl")
jsonld_file     = os.path.join(base_dir, f"{model_name}.jsonld")

# Load files
a = load_as_dict(summary_json)
b = load_as_dict(turtle_file)
c = load_as_dict(summary_json)
d = load_as_dict(jsonld_file)

# Perform comparisons
diffs_jsonld_vs_ttl = compare_json(a, b)
diffs_json_vs_jsonld = compare_json(c, d)

# Build DataFrames for interactive inspection
df1 = pd.DataFrame(diffs_jsonld_vs_ttl)
df2 = pd.DataFrame(diffs_json_vs_jsonld)

# --- Summaries & Filtering ---------------------------------------

def summarize_and_preview(df, preview_n=10):
    print("Change summary:")
    print(df['type'].value_counts().to_string(), "\n")
    
    print(f"First {preview_n} ‘changed’ entries:")
    # print(df[df['type']=="changed"].head(preview_n).to_string(index=False), "\n")
    
    # Top‐level (one slash) adds/removes
    top = df[df['path'].str.count("/") == 1]
    print("Top-level adds/removes:")
    print(top[top['type'].isin(['added','removed'])].to_string(index=False))

print("== JSON-LD vs TTL ==")
summarize_and_preview(df1)

print("\n== JSON vs JSON-LD ==")
summarize_and_preview(df2)



== JSON-LD vs TTL ==
Change summary:
type
changed    1 

First 10 ‘changed’ entries:
Top-level adds/removes:
Empty DataFrame
Columns: [path, type, a, b]
Index: []

== JSON vs JSON-LD ==
Change summary:
type
added      3
removed    1
changed    1 

First 10 ‘changed’ entries:
Top-level adds/removes:
Empty DataFrame
Columns: [path, type, a, b]
Index: []


In [68]:
# show all the removed paths (in JSON but not in JSON-LD)
print("Removed in JSON-LD comparison:")
print(df2[df2['type']=="removed"][['path']].to_string(index=False))

# show all the added paths (in JSON-LD but not in JSON)
print("\nAdded in JSON-LD comparison:")
print(df2[df2['type']=="added"][['path']].to_string(index=False))

Removed in JSON-LD comparison:
    path
end_time

Added in JSON-LD comparison:
     path
 @context
     used
generated


In [69]:
# show all the removed paths (in JSON but not in JSON-LD)
print("Removed in .ttl comparison:")
print(df1[df1['type']=="removed"][['path']].to_string(index=False))

# show all the added paths (in JSON-LD but not in JSON)
print("\nAdded in .ttl comparison:")
print(df1[df1['type']=="added"][['path']].to_string(index=False))

Removed in .ttl comparison:
Empty DataFrame
Columns: [path]
Index: []

Added in .ttl comparison:
Empty DataFrame
Columns: [path]
Index: []


Checks for completeness and mapping and time taken, needs work #TODO

In [70]:

# ── User configuration ─────────────────────────────────────────────────────────

# Which keys must appear in every run_summary.json?
REQUIRED_TOPLEVEL = {
    "run_id", "start_time", "end_time",
    "params", "metrics", "tags", "artifacts"
}

# A couple of sub-fields we also want to spot-check:
REQUIRED_PARAMS  = {"random_state"}
REQUIRED_METRICS = {"accuracy"}

JSON_SUMMARIES = glob.glob("MODEL_PROVENANCE/*_run_summary.json")


# ── Helpers ────────────────────────────────────────────────────────────────────

def iso8601(ms):
    return datetime.fromtimestamp(ms/1000, tz=timezone.utc).isoformat()


def load_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def write_json(path, obj):
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2)


def convert_to_jsonld_and_ttl(summary, basename):
    # build @context
    ctx = {
        "prov":    "http://www.w3.org/ns/prov#",
        "xsd":     "http://www.w3.org/2001/XMLSchema#",
        "run":     "prov:Activity",
        "start":   "prov:startedAtTime",
        "end":     "prov:endedAtTime",
        "used":    "prov:used",
        "gen":     "prov:generated",
        "param":   "prov:hadParameter",
        "metric":  "prov:hadQuality",
        "entity":  "prov:Entity",
        "label":   "prov:label",
        "value":   "prov:value",
        "version": "prov:hadRevision",
        "id":      "@id",
        "type":    "@type"
    }

    jsonld = {
        "@context": ctx,
        "@id":      f"urn:run:{summary['run_id']}",
        "@type":    "run",
        "start": {
            "@type":  "xsd:dateTime",
            "@value": iso8601(summary["start_time"])
        }
    }
    if summary.get("end_time") is not None:
        jsonld["end"] = {
            "@type":  "xsd:dateTime",
            "@value": iso8601(summary["end_time"])
        }

    # params
    jsonld["param"] = [
        {"@type":"entity","label":k,"value":str(v)}
        for k,v in summary.get("params",{}).items()
    ]
    # metrics
    jsonld["metric"] = [
        {"@type":"entity","label":k,
         "value":{"@type":"xsd:decimal","@value":v}}
        for k,v in summary.get("metrics",{}).items()
    ]
    # artifacts
    jsonld["gen"] = [
        {
            "@type":"entity",
            "label": art.get("path") or art.get("label"),
            "prov:location": (
                art.get("uri")
                or (art.get("content","")[:30]+"…")
                if isinstance(art.get("content"),str)
                else ""
            )
        }
        for art in summary.get("artifacts",[])
    ]
    # dataset used
    jsonld["used"] = {
        "@type":"entity",
        "label": summary["tags"].get("dataset_name"),
        "version": summary["tags"].get("dataset_version")
    }

    # write JSON-LD
    out_jsonld = f"MODEL_PROVENANCE/{basename}.jsonld"
    write_json(out_jsonld, jsonld)

    # serialize TTL
    g = Graph().parse(data=json.dumps(jsonld), format="json-ld")
    out_ttl = f"MODEL_PROVENANCE/{basename}.ttl"
    g.serialize(destination=out_ttl, format="turtle")

    return out_jsonld, out_ttl


def normalize_jsonld(js):
    """Simple deep-sort so compare_json doesn’t trip over ordering."""
    if isinstance(js, dict):
        return {k: normalize_jsonld(js[k]) for k in sorted(js)}
    if isinstance(js, list):
        return sorted((normalize_jsonld(el) for el in js),
                      key=lambda x: json.dumps(x, sort_keys=True))
    return js


def diff_roundtrip(orig_json, jsonld_path, ttl_path):
    orig = load_json(orig_json)
    ld   = load_json(jsonld_path)

    # parse TTL back to JSON-LD
    g = Graph().parse(ttl_path, format="turtle")
    ttl_as_ld = json.loads(g.serialize(format="json-ld"))

    # normalize
    nl = normalize_jsonld(ld)
    nt = normalize_jsonld(ttl_as_ld)

    return {
        "orig_vs_jsonld":   compare_json(orig, ld),
        "jsonld_vs_ttl_ld": compare_json(nl, nt)
    }


# ── Main flow ─────────────────────────────────────────────────────────────────

def main():
    ok = 0
    total = len(JSON_SUMMARIES)
    missing_reports = []
    cases = {}  # store diff results per run

    for js_path in JSON_SUMMARIES:
        summary = load_json(js_path)
        base    = os.path.basename(js_path).split("_run_summary.json")[0]

        # 1) completeness check
        if not REQUIRED_TOPLEVEL.issubset(summary):
            missing = REQUIRED_TOPLEVEL - set(summary)
            missing_reports.append((js_path, f"missing fields {missing}"))
            continue

        if not (REQUIRED_PARAMS <= summary["params"].keys()):
            missing_reports.append((js_path, f"params missing {REQUIRED_PARAMS - summary['params'].keys()}"))
            continue

        if not (REQUIRED_METRICS <= summary["metrics"].keys()):
            missing_reports.append((js_path, f"metrics missing {REQUIRED_METRICS - summary['metrics'].keys()}"))
            continue

        ok += 1

        # 2) convert
        jsonld_path, ttl_path = convert_to_jsonld_and_ttl(summary, base)

        # 3) diff
        diffs = diff_roundtrip(js_path, jsonld_path, ttl_path)
        cases[base] = diffs
        print(f"\n── {base} diffs ──")
        print("  • JSON → JSON-LD:", len(diffs["orig_vs_jsonld"]), "differences")
        print("  • JSON-LD → TTL → JSON-LD:", len(diffs["jsonld_vs_ttl_ld"]), "differences")

    # 4) completeness summary
    completeness_pct = (100 * ok / total) if total else 0
    print(f"\n{ok}/{total} runs passed completeness checks ({completeness_pct:.1f}%).")
    if missing_reports:
        print("\nFailures:")
        for path, reason in missing_reports:
            print(f" • {path}: {reason}")

    # 5) integrity check
    total_runs = len(cases)
    zero_diff_runs = sum(
        1
        for diffs in cases.values()
        if not diffs["orig_vs_jsonld"] and not diffs["jsonld_vs_ttl_ld"]
    )
    integrity_pct = (100 * zero_diff_runs / total_runs) if total_runs else 0
    print(f"\nMapping integrity: {zero_diff_runs}/{total_runs} runs have zero diffs — {integrity_pct:.1f}%")

    # 6) overall quality score
    overall_score = (completeness_pct + integrity_pct) / 2
    print(f"Overall quality score: {overall_score:.1f}%")

    # 7) Benchmark your training fn
    print("\nBenchmarking train_and_log() overhead:")
    def train_and_log(use_mlflow=False):
        # ← your real instrumentation + fit logic here
        time.sleep(0.5 + (0.1 if use_mlflow else 0))  # stub
        return

    for flag in (False, True):
        start = time.time()
        train_and_log(use_mlflow=flag)
        elapsed = time.time() - start
        label = "With MLflow" if flag else "No MLflow"
        print(f"  • {label:10s}: {elapsed:.3f}s")


if __name__ == "__main__":
    main()


0/0 runs passed completeness checks (0.0%).

Mapping integrity: 0/0 runs have zero diffs — 0.0%
Overall quality score: 0.0%

Benchmarking train_and_log() overhead:
  • No MLflow : 0.502s
  • With MLflow: 0.601s


RQ2  implementation

In [72]:

# Load all run summary JSON files
files = glob.glob("MODEL_PROVENANCE/*/*_run_summary.json")
rows = []
for f in files:
    with open(f) as fh:
        summary = json.load(fh)
    # Flatten parameters and metrics
    row = {"run_id": summary["run_id"]}
    row.update({f"param_{k}": v for k, v in summary.get("params", {}).items()})
    row.update({f"metric_{k}": v for k, v in summary.get("metrics", {}).items()})
    row.update({f"tag_{k}": v for k, v in summary.get("tags", {}).items()})
    rows.append(row)

# Create DataFrame
df = pd.DataFrame(rows)

# Display the DataFrame
df.columns


Index(['run_id', 'param_bootstrap', 'param_ccp_alpha', 'param_class_weight',
       'param_columns_raw', 'param_criterion', 'param_database.description',
       'param_database.id', 'param_database.name', 'param_database.owner',
       'param_dataset.authors', 'param_dataset.doi', 'param_dataset.published',
       'param_dataset.publisher', 'param_dataset.title',
       'param_dropped_columns', 'param_feature_names',
       'param_matplotlib_version', 'param_max_depth', 'param_max_features',
       'param_max_leaf_nodes', 'param_max_samples',
       'param_min_impurity_decrease', 'param_min_samples_leaf',
       'param_min_samples_split', 'param_min_weight_fraction_leaf',
       'param_numpy_version', 'param_n_estimators', 'param_n_features',
       'param_n_features_final', 'param_n_jobs', 'param_n_records',
       'param_n_test_samples', 'param_n_train_samples', 'param_oob_score',
       'param_os_platform', 'param_pandas_version', 'param_python_version',
       'param_random_state',

1) Tracing preprocessing steps
:
Here are the top 4 Iris‐focused preprocessing‐tracing use cases I’d tackle first:

Reconstruct a run’s exact preprocessing
Fetch a run’s run_id, columns_raw, dropped_columns, feature_names and test_size so you can replay the exact data pull & split.

Feature‐drop impact analysis
Identify runs where one or more measurements (e.g. petalwidthcm) were dropped and compare their test accuracies.

Best feature subset discovery
Group runs by which features they used (sepals only vs petals only vs both) and rank them by test F1 or accuracy.

Common steps in high-accuracy runs
Filter for runs with accuracy_score_X_test ≥ 0.95 and list the shared preprocessing settings (dropped columns, test_size, etc.).

In [73]:


# Helper to get the “official” feature_names from your summary DF
def _get_all_features(df):
    # assumes every row has the same param_feature_names
    raw = df.loc[0, 'param_feature_names']
    return ast.literal_eval(raw)

# Train & eval RF on just these columns of Iris
def evaluate_subset(features, test_size=0.2, random_state=42, n_estimators=200):
    iris = load_iris()
    X = pd.DataFrame(iris.data, columns=iris.feature_names)
    # map sklearn’s names to your param names, e.g. "sepal length (cm)" → "sepallengthcm"
    canon = _get_all_features(df)
    mapping = dict(zip(iris.feature_names, canon))
    X = X.rename(columns=mapping)
    X_sub = X[features]
    y = iris.target
    Xtr, Xte, ytr, yte = train_test_split(X_sub, y, test_size=test_size, random_state=random_state)
    m = RandomForestClassifier(n_estimators=n_estimators, random_state=random_state)
    m.fit(Xtr, ytr)
    return accuracy_score(yte, m.predict(Xte))
def trace_preprocessing(df, run_id=None):
    cols = ['run_id',
            'param_dataset.title',
            'param_columns_raw',
            'param_dropped_columns',
            'param_feature_names',
            'param_dataset.authors', 'param_dataset.doi', 'param_dataset.published',
            'param_test_size',
            'param_criterion',
            'param_max_depth','param_max_leaf_nodes', 'param_max_samples',
           'metric_accuracy','metric_f1_macro','metric_roc_auc']
    if run_id is None:
        subset = df.loc[:, cols]
    else:
        subset = df.loc[df['run_id'] == run_id, cols]
    return subset.to_dict(orient='records')


def drop_impact(df, feature, **_):
    all_feats = _get_all_features(df)
    baseline = evaluate_subset(all_feats)
    without = [f for f in all_feats if f!=feature]
    dropped = evaluate_subset(without)
    return {
      'dropped_feature': feature,
      'baseline_acc': baseline,
      'dropped_acc': dropped,
      'impact': baseline - dropped
    }

def drop_impact_all(df: pd.DataFrame) -> List[Dict[str, Any]]:
    """
    Compute drop-impact for every feature in the dataset.
    Returns list of dicts with dropped_feature, baseline_acc, dropped_acc, impact.
    """
    feats = _get_all_features(df)
    baseline = evaluate_subset(feats)
    summary = []
    for feat in feats:
        without = [f for f in feats if f != feat]
        acc = evaluate_subset(without)
        summary.append({
            'dropped_feature': feat,
            'baseline_acc': baseline,
            'dropped_acc': acc,
            'impact': round(baseline - acc, 4)
        })
    return summary

def best_feature_subset(df, features, **_):
    acc = evaluate_subset(features)
    return {'features': features, 'accuracy': acc}

def common_high_accuracy(df: pd.DataFrame, threshold: float = 0.95) -> List[Dict[str, Any]]:
    """
    Filter runs with test accuracy >= threshold and list unique shared preprocessing settings.
    """
    high = df[df['metric_accuracy_score_X_test'] >= threshold]
    cols = ['param_dropped_columns', 'param_test_size', 'param_feature_names']
    return high[cols].drop_duplicates().to_dict(orient='records')


# --------------------------------------------
# Use Case Registry with parameter order for minimal input
# --------------------------------------------
USE_CASES = {
    'trace_preprocessing': {
        'func': trace_preprocessing,
        'required_params': [],            # none strictly required
        'optional_params': ['run_id'],    # run_id can be supplied or not
    },
    'drop_impact': {
        'func': drop_impact,
        'required_params': ['feature'],
        'optional_params': [],
    },
     'drop_impact_all': {
        'func': drop_impact_all,
        'required_params': [],
        'optional_params': [],
    },
    'best_feature_subset': {
        'func': best_feature_subset,
        'required_params': ['features'],
        'optional_params': [],
    },
    'common_high_accuracy': {
        'func': common_high_accuracy,
        'required_params': ['threshold'],
        'optional_params': [],
    },
}


def call_use_case(df, use_case_name, **kwargs):
    if use_case_name not in USE_CASES:
        raise ValueError(f"Unknown use case: {use_case_name}")
    case = USE_CASES[use_case_name]
    func = case['func']
    # check required
    missing = [p for p in case['required_params'] if p not in kwargs]
    if missing:
        raise ValueError(f"{use_case_name} missing required params: {missing}")
    # build args
    args = {p: kwargs[p] for p in case['required_params']}
    for p in case['optional_params']:
        args[p] = kwargs.get(p)
    return func(df, **args)

# --------------------------------------------
# Example Usage
# --------------------------------------------
if __name__ == '__main__':
   # # 1) trace_preprocessing for all runs
    print(call_use_case(df, 'trace_preprocessing'))
    
    # 2) trace_preprocessing for a single run_id
    print(call_use_case(df, 'trace_preprocessing', run_id='361daa12f99f4129a06cd20b78dd6fa7'))

    # 5) common_high_accuracy
    print(call_use_case(df, 'common_high_accuracy', threshold=0.99))

    # 4) Best‐subset on just sepals:
    print(call_use_case(df, 'best_feature_subset', features=['sepallengthcm','sepalwidthcm']))

    # 3) Drop‐impact for “petallengthcm”:
    print(call_use_case(df, 'drop_impact', feature='petallengthcm'))

    print(call_use_case(df, 'drop_impact_all'))  # summary for all features


2025/04/25 12:23:44 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'df84c36b36cc4ebd90a999db3ebc4ad4', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


[{'run_id': '28f01e38b7f04d2f948fe21f57f41d0c', 'param_dataset.title': 'Scikit-Learn Iris', 'param_columns_raw': "['id', 'sepallengthcm', 'sepalwidthcm', 'petallengthcm', 'petalwidthcm', 'species']", 'param_dropped_columns': "['id']", 'param_feature_names': "['sepallengthcm', 'sepalwidthcm', 'petallengthcm', 'petalwidthcm']", 'param_dataset.authors': '["Marshall Michael"]', 'param_dataset.doi': '10.5281/ZENODO.1404173', 'param_dataset.published': '2018-8-27', 'param_test_size': '0.2', 'param_criterion': 'entropy', 'param_max_depth': '12', 'param_max_leaf_nodes': 'None', 'param_max_samples': 'None', 'metric_accuracy': 1.0, 'metric_f1_macro': 1.0, 'metric_roc_auc': 1.0}]
[]
[{'param_dropped_columns': "['id']", 'param_test_size': '0.2', 'param_feature_names': "['sepallengthcm', 'sepalwidthcm', 'petallengthcm', 'petalwidthcm']"}]


2025/04/25 12:23:52 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '41261519e1a643c5b1335701aee1bf95', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


{'features': ['sepallengthcm', 'sepalwidthcm'], 'accuracy': 0.7666666666666667}


2025/04/25 12:23:58 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e0955d231fa6488e9339086b5845064c', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


...

2025/04/25 12:24:10 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '8ca4591a1b53402f854187104d1e7ee0', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


2025/04/25 12:24:16 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '0c8a66e5e4b244f9a6a8e9fa02d26828', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


2025/04/25 12:24:22 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '53994143a51e481abd23e988be2466b1', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


2025/04/25 12:24:28 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '35588f1cd8c34ce28770848de714d3c4', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


[{'dropped_feature': 'sepallengthcm', 'baseline_acc': 1.0, 'dropped_acc': 1.0, 'impact': 0.0}, {'dropped_feature': 'sepalwidthcm', 'baseline_acc': 1.0, 'dropped_acc': 1.0, 'impact': 0.0}, {'dropped_feature': 'petallengthcm', 'baseline_acc': 1.0, 'dropped_acc': 0.9666666666666667, 'impact': 0.0333}, {'dropped_feature': 'petalwidthcm', 'baseline_acc': 1.0, 'dropped_acc': 0.9666666666666667, 'impact': 0.0333}]


 • Detecting models trained with deprecated code versions
 • Mapping models to specific datasets used during training

In [74]:

def detect_deprecated_code(df: pd.DataFrame, deprecated_commits: List[str], **_) -> List[Dict[str, Any]]:
    # we know the column is called tag_git_current_commit_hash
    commit_col = 'tag_git_current_commit_hash'
    if commit_col not in df.columns:
        raise KeyError(f"Missing {commit_col} in DataFrame")
    out = df[df[commit_col].isin(deprecated_commits)]
    # include run_id and notebook/runName for context
    cols = ['run_id', commit_col, 'tag_notebook_name', 'tag_mlflow.runName']
    # drop any that don’t exist
    cols = [c for c in cols if c in df.columns]
    return out[cols].to_dict(orient='records')


def map_model_dataset(df: pd.DataFrame, **_) -> List[Dict[str, Any]]:
    """
    For each run, return its model name (or run_id) alongside the dataset
    title, DOI, published date and publisher.
    """
    # pick whichever model-name column you have
    model_col = 'tag_model_name' if 'tag_model_name' in df.columns else 'param_model_name'
    cols = [
        'run_id',
        model_col,
        'param_dataset.title',
        'param_dataset.doi',
        'param_dataset.published',
        'param_dataset.publisher'
    ]
    # filter out any columns that don’t actually exist
    cols = [c for c in cols if c in df.columns]
    return df[cols].to_dict(orient='records')

# --------------------------------------------
# Extend Use-Case Registry
# --------------------------------------------
USE_CASES.update({
    'detect_deprecated_code': {
        'func': detect_deprecated_code,
        'required_params': ['deprecated_commits'],
        'optional_params': []
    },
    'map_model_dataset': {
        'func': map_model_dataset,
        'required_params': [],
        'optional_params': []
    },
})
# 1) Detect runs on deprecated commits:
deprecated = [
    "a07434af4f547af2daab044d6873eb7081162293",
    "d329c92495e196ec0f39fbb19dfdd367131a77d9"
]
# print(call_use_case(df, "detect_deprecated_code", deprecated_commits=deprecated))
pprint(call_use_case(df, 'map_model_dataset'))


[{'param_dataset.doi': '10.5281/ZENODO.1404173',
  'param_dataset.published': '2018-8-27',
  'param_dataset.publisher': 'Zenodo',
  'param_dataset.title': 'Scikit-Learn Iris',
  'run_id': '28f01e38b7f04d2f948fe21f57f41d0c',
  'tag_model_name': 'RandomForest_Iris_v20250425_121328'}]


Goal: Notify collaborators who have forked the GitHub repo if their fork is outdated (i.e., behind the current commit used to train a model).

🧠 What We Need
Current training run’s Git commit hash

GitHub API to fetch all forks of your repo

Compare each fork’s main or master branch head commit

Create an issue on their fork or on your repo tagging them if they’re behind

: Notify via issues on your own repo

In [75]:
def notify_outdated_forks():
    load_dotenv()
    token     = os.getenv("THESIS_TOKEN")
    owner     = "reema-dass26"
    repo      = "REPO"

    if not token:
        print("⚠️ GITHUB_TOKEN not set.")
        return

    headers = {
        "Authorization": f"token {token}",
        "Accept":        "application/vnd.github.v3+json"
    }

    # 1) Get latest upstream commit
    main_commits = requests.get(
        f"https://api.github.com/repos/{owner}/{repo}/commits",
        headers=headers,
        params={"per_page": 1}
    )
    main_commits.raise_for_status()
    new_commit_hash = main_commits.json()[0]["sha"]
    print(f"Latest upstream commit: {new_commit_hash}")

    # 2) List forks
    forks_resp = requests.get(f"https://api.github.com/repos/{owner}/{repo}/forks", headers=headers)
    forks_resp.raise_for_status()
    forks = forks_resp.json()

    # 3) Compare each fork
    outdated = []
    for fork in forks:
        fork_owner = fork["owner"]["login"]
        fork_comm = requests.get(
            fork["url"] + "/commits",
            headers=headers,
            params={"per_page": 1}
        )
        if fork_comm.status_code != 200:
            print(f"  – could not fetch commits for {fork_owner}, skipping.")
            continue

        fork_sha = fork_comm.json()[0]["sha"]
        if fork_sha != new_commit_hash:
            outdated.append(f"@{fork_owner}")

    # 4) Open an issue if any are behind
    if outdated:
        title = "🔔 Notification: Your fork is behind the latest commit"
        body  = (
            f"Hi {' '.join(outdated)},\n\n"
            f"The main repository has been updated to commit `{new_commit_hash}`.\n"
            "Please consider pulling the latest changes to stay in sync.\n\n"
            "Thanks!"
        )
        issues_url = f"https://api.github.com/repos/{owner}/{repo}/issues"
        resp = requests.post(
        issues_url,
        headers=headers,
        json={"title": title, "body": body}
    )

    # DEBUGGING OUTPUT
    print(f"→ POST {issues_url}")
    print("→ Status code:", resp.status_code)
    print("→ Response headers:", resp.headers)
    try:
        data = resp.json()
        print("→ Response JSON:", data)
        print("→ html_url field:", data.get("html_url"))
    except ValueError:
        print("→ No JSON response body; raw text:", resp.text)

if __name__ == "__main__":
    answer = input("Do you want to notify collaborators whose forks are behind? (y/N): ").strip().lower()
    if answer in ("y", "yes"):
        notify_outdated_forks()
    else:
        print("No action taken.")


Do you want to notify collaborators whose forks are behind? (y/N):  N


No action taken.


INVENIO INTEGRETION to upload the necessary files and publish

In [ ]:
############################################################################################
# TEST CODE FOR INVENIO INTEGRETION
#############################################################################################




# API_BASE = "https://127.0.0.1:5000"
# TOKEN    = "8LnqJuz3TsBHffnDJ3isPLHYHtRbWrC0M667Nb5haEbnXpWqGbFRyfDApymr"

# # 1) Test read‐scope by listing records (no size param or size=1)
# resp = requests.get(
#     f"{API_BASE}/api/records",
#     headers={"Authorization": f"Bearer {TOKEN}"},
#     verify=False
# )
# print(resp.status_code)
# # should be 200 and a JSON page of records

# # or explicitly:
# resp = requests.get(
#     f"{API_BASE}/api/records?size=1",
#     headers={"Authorization": f"Bearer {TOKEN}"},
#     verify=False
# )
# print(resp.status_code, resp.json())
# #################################################################################################
# API_BASE = "https://127.0.0.1:5000"
# TOKEN    = "8LnqJuz3TsBHffnDJ3isPLHYHtRbWrC0M667Nb5haEbnXpWqGbFRyfDApymr"

# resp = requests.options(
#     f"{API_BASE}/api/records",
#     headers={"Authorization": f"Bearer {TOKEN}"},
#     verify=False
# )
# print("Allowed methods:", resp.headers.get("Allow"))

In [ ]:

# -----------------------------------------------------------------------------
# Configuration
# -----------------------------------------------------------------------------
API_BASE   = "https://127.0.0.1:5000"
TOKEN      = "8LnqJuz3TsBHffnDJ3isPLHYHtRbWrC0M667Nb5haEbnXpWqGbFRyfDApymr"
VERIFY_SSL = False  # only for self‐signed dev

HEADERS_JSON = {
    "Accept":        "application/json",
    "Content-Type":  "application/json",
    "Authorization": f"Bearer {TOKEN}",
}

HEADERS_OCTET = {
    "Content-Type":  "application/octet-stream",
    "Authorization": f"Bearer {TOKEN}",
}

# The folders you want to walk & upload:
TO_UPLOAD = ["Trained_models", "plots", "MODEL_PROVENANCE"]


# -----------------------------------------------------------------------------
# 1) Create draft with ALL required metadata
# -----------------------------------------------------------------------------
def create_draft():
    payload = {
  "metadata": {
    "title":            "RandomForest Iris Model Artifacts",
    "creators": [ {
      "person_or_org": {
        "type":        "personal",
        "given_name":  "Reema",
        "family_name": "Dass"
      }
    } ],
    "publication_date": "2025-04-24",
    "resource_type":    { "id": "software" },
    "access": {
      "record": "public",
      "files":  "public"
    }
  }
}
    r = requests.post(f"{API_BASE}/api/records",
                      headers=HEADERS_JSON,
                      json=payload,
                      verify=VERIFY_SSL)
    r.raise_for_status()
    draft = r.json()
    print("✅ Draft created:", draft["id"])
    return draft["id"], draft["links"]


# -----------------------------------------------------------------------------
# 2) Register, upload and commit a single file
# -----------------------------------------------------------------------------
def upload_and_commit(links, key, path):
    # 2a) register the filename in the draft
    r1 = requests.post(links["files"],
                       headers=HEADERS_JSON,
                       json=[{"key": key}],
                       verify=VERIFY_SSL)
    r1.raise_for_status()
    entry = next(e for e in r1.json()["entries"] if e["key"] == key)
    file_links = entry["links"]

    # 2b) upload the bytes
    with open(path, "rb") as fp:
        r2 = requests.put(file_links["content"],
                          headers=HEADERS_OCTET,
                          data=fp,
                          verify=VERIFY_SSL)
    r2.raise_for_status()

    # 2c) commit the upload
    r3 = requests.post(file_links["commit"],
                       headers=HEADERS_JSON,
                       verify=VERIFY_SSL)
    r3.raise_for_status()
    print(f"  • Uploaded {key}")


# -----------------------------------------------------------------------------
# 3) Walk each folder and upload every file
# -----------------------------------------------------------------------------
def upload_folder(links):
    for folder in TO_UPLOAD:
        if not os.path.isdir(folder):
            print(f"⚠️ Skipping missing folder {folder}")
            continue
        base = os.path.dirname(folder) or folder
        for root, _, files in os.walk(folder):
            for fn in files:
                local = os.path.join(root, fn)
                # create a POSIX‐style key preserving subfolders
                key = os.path.relpath(local, start=base).replace(os.sep, "/")
                upload_and_commit(links, key, local)


# -----------------------------------------------------------------------------
# 4) Publish the draft
# -----------------------------------------------------------------------------
def publish(links):
    r = requests.post(links["publish"],
                      headers=HEADERS_JSON,
                      verify=VERIFY_SSL)
    if not r.ok:
        print("❌ Publish failed:", r.status_code, r.text)
        try: print(r.json())
        except: pass
        r.raise_for_status()
    print("✅ Published:", r.json()["id"])


# -----------------------------------------------------------------------------
# Main
# -----------------------------------------------------------------------------
if __name__ == "__main__":
    recid, links = create_draft()
    upload_folder(links)
    publish(links)

  


########################################################################
# FETCH metadata from INVENIO
########################################################################

In [96]:
def fetch_metadata(record_id, model_name, api_base, headers, verify_ssl=True):
    """
    Fetch Invenio metadata and save to a file named after the model inside 'Invenio_metadata' folder.
    """
    response = requests.get(f"{API_BASE}/api/records/{record_id}",
                            headers=headers,
                            verify=VERIFY_SSL)
    response.raise_for_status()
    metadata = response.json()

    print("✅ Metadata fetched successfully")

    # Create the folder if it doesn't exist
    os.makedirs("Invenio_metadata", exist_ok=True)

    # Construct path and save
    file_path = os.path.join("Invenio_metadata", f"{model_name}_invenio.json")
    with open(file_path, "w") as f:
        json.dump(metadata, f, indent=4)

    print(f"✅ Metadata saved at {file_path}")
    return file_path
path = fetch_metadata(recid, model_name, api_base=API_BASE, headers=HEADERS_JSON)
print(path)

C:\Users\reema\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


✅ Metadata fetched successfully
✅ Metadata saved at Invenio_metadata\RandomForest_Iris_v20250425_135900_invenio.json
Invenio_metadata\RandomForest_Iris_v20250425_135900_invenio.json


METADATA EXTRACTION FROM INVENIO and ADD it to main Provenence FILE

In [97]:
# ----------------------------
# Function: Extract metadata
# ----------------------------
def extract_metadata(metadata):
    print("✅ Metadata loaded successfully")
    print("ℹ️ ID:", metadata.get("id", "N/A"))
    print("🔍 Extracting required fields...")

    extracted_data = {
        "invenio_metadata": {
            "id": metadata.get("id", ""),
            "title": metadata.get("metadata", {}).get("title", ""),
            "creator": ", ".join([
                creator["person_or_org"].get("name", "")
                for creator in metadata.get("metadata", {}).get("creators", [])
            ]),
            "publication_date": metadata.get("metadata", {}).get("publication_date", ""),
            "files": [],
            "pids": metadata.get("pids", {}),
            "version_info": metadata.get("versions", {}),
            "status": metadata.get("status", ""),
            "views": metadata.get("stats", {}).get("this_version", {}).get("views", 0),
            "downloads": metadata.get("stats", {}).get("this_version", {}).get("downloads", 0),
        }
    }

    for key, file_info in metadata.get("files", {}).get("entries", {}).items():
        file_detail = {
            "key": key,
            "url": file_info["links"].get("content", ""),
            "size": file_info.get("size", 0),
            "mimetype": file_info.get("mimetype", ""),
            "checksum": file_info.get("checksum", ""),
            "metadata": file_info.get("metadata", {}),
        }
        extracted_data["invenio_metadata"]["files"].append(file_detail)

    return extracted_data


invenio_path = f"Invenio_metadata/{model_name}_invenio.json"
run_summary_path = f"MODEL_PROVENANCE/{model_name}/{model_name}_run_summary.json"

# ----------------------------
# Step 1: Load Invenio metadata
# ----------------------------
with open(invenio_path, "r") as f:
    original_metadata = json.load(f)

# ----------------------------
# Step 2: Extract metadata
# ----------------------------
extracted_metadata = extract_metadata(original_metadata)
print("📤 Extracted Metadata Preview:")
print(json.dumps(extracted_metadata, indent=4)[:1000])  # Preview

# ----------------------------
# Step 3: Load run summary
# ----------------------------
with open(run_summary_path, "r") as f:
    existing_metadata = json.load(f)

# ----------------------------
# Step 4: Merge metadata
# ----------------------------
existing_metadata.update(extracted_metadata)

# ----------------------------
# Step 5: Save updated summary
# ----------------------------
with open(run_summary_path, "w") as f:
    json.dump(existing_metadata, f, indent=4)

print(f"✅ Invenio metadata embedded successfully into: {run_summary_path}")

✅ Metadata loaded successfully
ℹ️ ID: 892h2-fq661
🔍 Extracting required fields...
📤 Extracted Metadata Preview:
{
    "invenio_metadata": {
        "id": "892h2-fq661",
        "title": "RandomForest Iris Model Artifacts",
        "creator": "Dass, Reema",
        "publication_date": "2025-04-24",
        "files": [
            {
                "key": "RandomForest_Iris_v20250423_230422.pkl",
                "url": "https://127.0.0.1:5000/api/records/892h2-fq661/files/RandomForest_Iris_v20250423_230422.pkl/content",
                "size": 282910,
                "mimetype": "application/octet-stream",
                "checksum": "md5:a9f9e15b9c808d94c8e5737089beaa7d",
                "metadata": {}
            },
            {
                "key": "RandomForest_Iris_v20250425_125653.pkl",
                "url": "https://127.0.0.1:5000/api/records/892h2-fq661/files/RandomForest_Iris_v20250425_125653.pkl/content",
                "size": 282910,
                "mimetype": "applicati

In [23]:
import subprocess

def get_git_version_info():
    try:
        commit = subprocess.check_output(["git", "rev-parse", "HEAD"]).decode().strip()
        tag = subprocess.check_output(["git", "describe", "--tags", "--exact-match"], stderr=subprocess.DEVNULL).decode().strip()
    except subprocess.CalledProcessError:
        tag = "untagged"
    return commit, tag

commit_hash, version_tag = get_git_version_info()
print("Commit:", commit_hash)
print("Version Tag:", version_tag)


Commit: 8f829893e0cbe7593ce4f796e53b019a0d1982c1
Version Tag: v2
